In [1]:
import os
import glob
import pickle
import numpy as np
from tqdm import tqdm
import torch.nn as nn
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

import sys, string, random
from datetime import datetime
from omegaconf import OmegaConf
import wandb
import torch
import lightning as L
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from torch.utils.data.sampler import Sampler, SubsetRandomSampler
from monai.networks.nets import UNet

sys.path.append('../')
from data_utils import MNMv2DataModule
from unet import LightningSegmentationModel
import torch.nn.functional as F

class ActualSequentialSampler(Sampler):
	r"""Samples elements sequentially, always in the same order.

	Arguments:
		data_source (Dataset): dataset to sample from
	"""

	def __init__(self, data_source):
		self.data_source = data_source

	def __iter__(self):
		return iter(self.data_source)

	def __len__(self):
		return len(self.data_source)
     
class CLUESampling:
    """
    Implements CLUE: Clustering via Uncertainty-weighted Embeddings for segmentation tasks.
    """
    def __init__(self, dset, model, device, args, balanced=False):
        self.dset = dset
        self.model = model
        self.device = device
        self.args = args
        self.random_state = np.random.RandomState(1234)
        self.T = args.clue_softmax_t
    
    def get_embedding(self, model, loader, device, args, with_emb=False):
        model.eval()
        embedding_pen = None
        embedding = None
        emb_dim = None
        batch_sz = args.batch_size
        num_samples = len(self.dset)

        with torch.no_grad():
            for batch_idx, batch in enumerate(tqdm(loader)):
                data = batch['data'].to(device)

                if with_emb:
                    e1, e2 = model(data, with_emb=True)
                    height, width = e2.shape[2], e2.shape[3]
                # else:
                #     e1 = model(data, with_emb=False)

                # Приводим размер логитов к размеру эмбеддингов
                e1 = F.interpolate(e1, size=(height, width), mode='bilinear', align_corners=False)

                if embedding_pen is None:
                    emb_dim = e2.shape[1]
                    num_classes = e1.shape[1]
                    embedding_pen = torch.zeros((num_samples * height * width, emb_dim), device='cpu')
                    embedding = torch.zeros((num_samples * height * width, num_classes), device='cpu')

                # Преобразуем логиты и эмбеддинги для каждого пикселя
                e1 = e1.permute(0, 2, 3, 1).reshape(-1, num_classes)
                e2 = e2.permute(0, 2, 3, 1).reshape(-1, emb_dim)

                # Вычисляем текущие индексы
                start_idx = batch_idx * batch_sz * height * width
                end_idx = start_idx + min(batch_sz * height * width, e2.shape[0])

                # Заполняем тензоры
                embedding[start_idx:end_idx, :] = e1.cpu()
                embedding_pen[start_idx:end_idx, :] = e2.cpu()

                if batch_idx > 1:
                    break
        
        return embedding, embedding_pen
    
    def query(self, n, data_loader):
        self.model.eval()

        # Получаем эмбеддинги для пикселей
        tgt_emb, tgt_pen_emb = self.get_embedding(self.model, data_loader, self.device, self.args, with_emb=True)

        # Используем предпоследние эмбеддинги (tgt_pen_emb)
        tgt_pen_emb = tgt_pen_emb.cpu().numpy()

        # Вычисляем неопределенность через энтропию для каждого пикселя
        tgt_scores = nn.Softmax(dim=1)(tgt_emb / self.T)
        tgt_scores += 1e-8
        sample_weights = -(tgt_scores * torch.log(tgt_scores)).sum(1).cpu().numpy()

        # Запуск K-means с учетом весов неопределенности
        km = KMeans(n)
        km.fit(tgt_pen_emb, sample_weight=sample_weights)

        # Центроиды эмбединги
        return km.cluster_centers_

if __name__ == '__main__':
    mnmv2_config   = OmegaConf.load('../../configs/mnmv2.yaml')
    unet_config    = OmegaConf.load('../../configs/monai_unet.yaml')
    trainer_config = OmegaConf.load('../../configs/unet_trainer.yaml')

    # init datamodule
    datamodule = MNMv2DataModule(
        data_dir=mnmv2_config.data_dir,
        vendor_assignment=mnmv2_config.vendor_assignment,
        batch_size=mnmv2_config.batch_size,
        binary_target=mnmv2_config.binary_target,
        non_empty_target=mnmv2_config.non_empty_target,
    )

    datamodule.setup(stage='fit')

    # init model
    cfg = OmegaConf.create({
        'unet_config': unet_config,
        'binary_target': True if unet_config.out_channels == 1 else False,
        'lr': unet_config.lr,
        'patience': unet_config.patience,
        'lambda_centroids': 0.6,
        'dataset': OmegaConf.to_container(mnmv2_config),
        'unet': OmegaConf.to_container(unet_config),
        'trainer': OmegaConf.to_container(trainer_config),
    })

    checkpoint_path = '../../pre-trained/trained_UNets/mnmv2-10-12_06-11-2024.ckpt'

    load_as_lightning_module = True#False
    load_as_pytorch_module = False#True

    if load_as_lightning_module:
        unet_config    = OmegaConf.load('../../configs/monai_unet.yaml')
        unet = UNet(
            spatial_dims=unet_config.spatial_dims,
            in_channels=unet_config.in_channels,
            out_channels=unet_config.out_channels,
            channels=[unet_config.n_filters_init * 2 ** i for i in range(unet_config.depth)],
            strides=[2] * (unet_config.depth - 1),
            num_res_units=4
        )
        
        model = LightningSegmentationModel.load_from_checkpoint(
            checkpoint_path,
            map_location=torch.device("cpu"),
            model=unet,
            binary_target=True if unet_config.out_channels == 1 else False,
            lr=unet_config.lr,
            patience=unet_config.patience,
            cfg=cfg
        )

    elif load_as_pytorch_module:
        checkpoint = torch.load(checkpoint_path, map_location=torch.device("cpu"))
        model_state_dict = checkpoint['state_dict']
        model_state_dict = {k.replace('model.model.', 'model.'): v for k, v in model_state_dict.items() if k.startswith('model.')}
        model_config = checkpoint['hyper_parameters']['cfgs']

        print(model_config)

        unet = UNet(
            spatial_dims=model_config['unet']['spatial_dims'],
            in_channels=model_config['unet']['in_channels'],
            out_channels=model_config['unet']['out_channels'],
            channels=[model_config['unet']['n_filters_init'] * 2 ** i for i in range(model_config['unet']['depth'])],
            strides=[2] * (model_config['unet']['depth'] - 1),
            num_res_units=4
        )

        unet.load_state_dict(model_state_dict)
    
    train_loader = datamodule.train_dataloader()
    device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    clue_sampler = CLUESampling(dset=datamodule.mnm_train, 
                                model=model, 
                                device=device, 
                                args=unet_config, )
                                            #cache_path='../../MedImSeg-Lab24/checkpoints/emb_and_weights.pkl')
    
    centroids = clue_sampler.query(n=2, data_loader=train_loader)

    datamodule.setup(stage='test')
    test_loader = datamodule.test_dataloader()
    start_loss  = model.test_model(test_loader, device)
    # out_str = '{} | Test performance on {}->{}: Round 0 (B=0): {:.2f}'.format(method, source, target, start_perf)

    # Шаг дообучения модели с использованием центроидов
    model.finetune_model_on_centroids(centroids, train_loader, model)

    # Тестируем производительность модели после дообучения
    test_perf = model.test_model(test_loader, device)
    # out_str += '\t Round 1 (B={}): {:.2f}'.format(len(cluster_centers), test_perf)

    # Выводим результаты и возвращаем производительность до и после обучения
    print(start_loss['dsc'], test_perf['dsc'])



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

nnUNet_raw_data_base is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
RESULTS_FOLDER is not defined and nnU-Net cannot be used for training or inference. If this is not in

/home/mikhelson/MedImSeg-Lab24/CLUE/lib/python3.8/site-packages/lightning/fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
2it [00:02,  1.24s/it]
/home/mikhelso

Test Results - Average Loss: 0.5151, Average Dice Score: 0.7595
{'unet_config': {'n_filters_init': 16, 'depth': 4, 'spatial_dims': 2, 'in_channels': 1, 'out_channels': 4, 'num_res_units': 4, 'lr': 0.001, 'patience': 5, 'clue_softmax_t': 1.0, 'batch_size': 32}, 'binary_target': False, 'lr': 0.001, 'patience': 5, 'lambda_centroids': 0.6, 'dataset': {'data_dir': '/home/mikhelson/MedImSeg-Lab24/data/MNM/', 'vendor_assignment': {'train': 'siemens', 'test': 'ge'}, 'batch_size': 32, 'binary_target': False, 'non_empty_target': False}, 'unet': {'n_filters_init': 16, 'depth': 4, 'spatial_dims': 2, 'in_channels': 1, 'out_channels': 4, 'num_res_units': 4, 'lr': 0.001, 'patience': 5, 'clue_softmax_t': 1.0, 'batch_size': 32}, 'trainer': {'train_transforms': 'global_transforms', 'limit_train_batches': 50, 'max_epochs': 100, 'early_stopping': {'monitor': 'val_loss', 'mode': 'min'}, 'model_checkpoint': {'save_top_k': 2, 'dirpath': '../../pre-trained/trained_UNets', 'monitor': 'val_loss'}, 'logging': Tr

0it [00:00, ?it/s]

tensor(5.0549, device='cuda:2', grad_fn=<MeanBackward0>) tensor(5.5875, device='cuda:2', grad_fn=<MeanBackward0>)
tensor(0.3130, device='cuda:2', grad_fn=<AddBackward0>) tensor(0.9047, device='cuda:2', grad_fn=<DivBackward0>)


...

894it [01:24, 10.52it/s]

tensor(0.9180, device='cuda:2', grad_fn=<MeanBackward0>) tensor(3.3766, device='cuda:2', grad_fn=<MeanBackward0>)
tensor(0.4140, device='cuda:2', grad_fn=<AddBackward0>) tensor(0.2719, device='cuda:2', grad_fn=<DivBackward0>)
tensor(0.9032, device='cuda:2', grad_fn=<MeanBackward0>) tensor(3.3692, device='cuda:2', grad_fn=<MeanBackward0>)
tensor(0.3573, device='cuda:2', grad_fn=<AddBackward0>) tensor(0.2681, device='cuda:2', grad_fn=<DivBackward0>)
tensor(0.8935, device='cuda:2', grad_fn=<MeanBackward0>) tensor(3.3575, device='cuda:2', grad_fn=<MeanBackward0>)
tensor(0.3547, device='cuda:2', grad_fn=<AddBackward0>) tensor(0.2661, device='cuda:2', grad_fn=<DivBackward0>)


897it [01:24, 10.63it/s]

tensor(0.8945, device='cuda:2', grad_fn=<MeanBackward0>) tensor(3.3489, device='cuda:2', grad_fn=<MeanBackward0>)
tensor(0.3612, device='cuda:2', grad_fn=<AddBackward0>) tensor(0.2671, device='cuda:2', grad_fn=<DivBackward0>)
tensor(0.8941, device='cuda:2', grad_fn=<MeanBackward0>) tensor(3.3529, device='cuda:2', grad_fn=<MeanBackward0>)
tensor(0.3768, device='cuda:2', grad_fn=<AddBackward0>) tensor(0.2667, device='cuda:2', grad_fn=<DivBackward0>)


KeyboardInterrupt: 

In [2]:
# # Шаг дообучения модели с использованием центроидов
# model.finetune_model_on_centroids(centroids, train_loader, model)

# # Тестируем производительность модели после дообучения
# test_perf = model.test_model(test_loader, device)
# # out_str += '\t Round 1 (B={}): {:.2f}'.format(len(cluster_centers), test_perf)

# # Выводим результаты и возвращаем производительность до и после обучения
# print(start_loss['dsc'], test_perf['dsc'])
cfg

{'unet_config': {'n_filters_init': 16, 'depth': 4, 'spatial_dims': 2, 'in_channels': 1, 'out_channels': 4, 'num_res_units': 4, 'lr': 0.001, 'patience': 5, 'clue_softmax_t': 1.0, 'batch_size': 32}, 'binary_target': False, 'lr': 0.001, 'patience': 5, 'lambda_centroids': 0.6, 'dataset': {'data_dir': '/home/mikhelson/MedImSeg-Lab24/data/MNM/', 'vendor_assignment': {'train': 'siemens', 'test': 'ge'}, 'batch_size': 32, 'binary_target': False, 'non_empty_target': False}, 'unet': {'n_filters_init': 16, 'depth': 4, 'spatial_dims': 2, 'in_channels': 1, 'out_channels': 4, 'num_res_units': 4, 'lr': 0.001, 'patience': 5, 'clue_softmax_t': 1.0, 'batch_size': 32}, 'trainer': {'train_transforms': 'global_transforms', 'limit_train_batches': 50, 'max_epochs': 100, 'early_stopping': {'monitor': 'val_loss', 'mode': 'min'}, 'model_checkpoint': {'save_top_k': 2, 'dirpath': '../../pre-trained/trained_UNets', 'monitor': 'val_loss'}, 'logging': True}}